# CIFAR Results
- CIFAR10/CIFAR100
- VGG-16/ResNet-56
- Results in paper are averaged over multiple runs

In [1]:
import pandas as pd
from rni.cifar import set_dataset_path, train, prune_finetune

In [9]:
set_dataset_path("cifar10", "/data/datasets/CIFAR10")
set_dataset_path("cifar100", "/data/datasets/CIFAR100")

# VGG-16

In [42]:
hyperparams = [
    {"method": "rni", "reg_weight": 1e-3, "b": 3},
    {"method": "l1", "reg_weight": 1e-4},
    {"method": "ucs"}
]

res_vgg = []
for hp in hyperparams:
    for ds in ["cifar10", "cifar100"]:
        trained_model, trained_logs = train(dataset=ds, arch="vgg16", **hp)
        pruned_model_50, pruned_logs_50 = prune_finetune(trained_model, dataset=ds, method=hp["method"], prune_pc=0.5)
        pruned_model_90, pruned_logs_90 = prune_finetune(trained_model, dataset=ds, method=hp["method"], prune_pc=0.9)

        res_vgg.append({
            **hp,
            "dataset": ds,
            "train_acc": trained_logs["test_acc"][trained_logs["best_epoch"]],
            "prune_acc_50": pruned_logs_50["test_acc"][pruned_logs_50["best_epoch"]],
            "prune_acc_90": pruned_logs_90["test_acc"][pruned_logs_90["best_epoch"]],
        })


res_vgg_df = pd.DataFrame(res_vgg).drop(["reg_weight", "b"], axis=1).set_index(["dataset", "method"]).sort_index()
res_vgg_df

train_acc  prune_acc_50  prune_acc_90
dataset  method                                       
cifar10  l1         0.9387        0.9367        0.8995
         rni        0.9363        0.9367        0.9088
         ucs        0.9358        0.9293        0.8405
cifar100 l1         0.7450        0.7134        0.2581
         rni        0.7201        0.7221        0.5935
         ucs        0.7371        0.7067        0.5460

# ResNet-56

In [43]:
hyperparams = [
    {"method": "rni", "reg_weight": 1e-4, "b": 0},
    {"method": "l1", "reg_weight": 1e-5},
    {"method": "ucs"}
]

res_resnet = []
for hp in hyperparams:
    for ds in ["cifar10", "cifar100"]:
        trained_model, trained_logs = train(dataset=ds, arch="resnet56", **hp)
        pruned_model_50, pruned_logs_50 = prune_finetune(trained_model, dataset=ds, method=hp["method"], prune_pc=0.5)
        pruned_model_90, pruned_logs_90 = prune_finetune(trained_model, dataset=ds, method=hp["method"], prune_pc=0.9)

        res_resnet.append({
            **hp,
            "dataset": ds,
            "train_acc": trained_logs["test_acc"][trained_logs["best_epoch"]],
            "prune_acc_50": pruned_logs_50["test_acc"][pruned_logs_50["best_epoch"]],
            "prune_acc_90": pruned_logs_90["test_acc"][pruned_logs_90["best_epoch"]],
        })


res_resnet_df = pd.DataFrame(res_resnet).drop(["reg_weight", "b"], axis=1).set_index(["dataset", "method"]).sort_index()
res_resnet_df

train_acc  prune_acc_50  prune_acc_90
dataset  method                                       
cifar10  l1         0.9403        0.9337        0.8991
         rni        0.9322        0.9326        0.9001
         ucs        0.9354        0.9351        0.8881
cifar100 l1         0.7167        0.7029        0.6192
         rni        0.7136        0.7038        0.6401
         ucs        0.7157        0.7114        0.6445